# CommonLit Readability Price : Simple Way

## Import Libraries

In [ ]:
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Load the train and test data
train_df = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
test_df = pd.read_csv("../input/commonlitreadabilityprize/test.csv")

result = pd.DataFrame()
result['id'] = test_df['id']

train_df = train_df[['excerpt', 'target']]
test_df = test_df[['excerpt']]

In [ ]:
test_df.head()

In [ ]:
train_df.head()

In [ ]:
train_df.iloc[1,0]

In [ ]:
# Create a preprocessing function to handle the text data
def preprocess(text):
    text = re.sub("[^a-zA-Z]" , " ", text)
    text = text.lower()
    
    # tokenize into words
    tokens = [word for sent in sent_tokenize(text) for word in word_tokenize(sent)]
    
    # remove punctuation
    interpunctuations = ['``', ',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
    tokens = [token for token in tokens if token not in interpunctuations]
    
    # lower capitalization
    tokens = [word.lower() for word in tokens]
    
    # lemmatize
    lemma = nltk.WordNetLemmatizer()
    tokens = [lemma.lemmatize(word) for word in tokens]
    preprocessed_text = " ".join(tokens)
    return preprocessed_text

train_df["excerpt"] = train_df["excerpt"].apply(preprocess)
test_df["excerpt"] = test_df["excerpt"].apply(preprocess)

In [ ]:
train_df.iloc[1,0]

In [ ]:
train_text = train_df[["excerpt"]]
test_text = test_df[["excerpt"]]
final_text = pd.concat([train_text , test_text])

In [ ]:
final_text["excerpt"][0]

In [ ]:
word_vectorizer = TfidfVectorizer()
word_vectorizer.fit(final_text["excerpt"])
train_word_features = word_vectorizer.transform(train_df["excerpt"])
test_word_features = word_vectorizer.transform(test_df["excerpt"])

In [ ]:
train_word_features.shape

In [ ]:
# Split the X_train , y_train ,X_test
X_train = train_word_features
y_train = train_df["target"]
X_test = test_word_features


# I use the XGBRegressor for our problem
xgb_model = XGBRegressor(learning_rate = 0.1,n_estimators = 200,
                        max_depth = 5, min_child_weight = 1,
                        gamma = 0 , subsample = 0.8,
                        colsample_bytree = 0.8,
                        objective = "reg:squarederror",
                        seed = 10)

xgb_model.fit(X_train , y_train)

## Predictions

In [ ]:
predictions = xgb_model.predict(X_test)
predictions[:10]

In [ ]:
result["target"] = predictions

In [ ]:
result.head()

In [ ]:
# make submission

result.to_csv("submission.csv" , index = False)